In [2]:
import os
import sys
import regex as re
from nltk.stem import PorterStemmer
sys.path.append("../")
from data_extraction.utils import data_extraction

ps = PorterStemmer()

In [3]:
data_path = '../data'
doc_id_tag = 'DOCNO'
doc_text_tag = 'TEXT'

# Loading stopwords

In [4]:
stopwords_path = '../stopwordlist.txt'
with open(stopwords_path, 'r') as file:
    stopwords = file.read()
    stopwords = stopwords.split('\n')
    stopwords_list = [word.strip().lower() for word in stopwords if word]

print(stopwords_list[:10])


['a', 'able', 'about', 'above', 'according', 'accordingly', 'across', 'actually', 'after', 'afterwards']


In [5]:
# extract text from all folders and files
extracted_text = {}

for folderID in os.listdir(data_path):
    extracted_text[folderID] = {}
    with open(os.path.join(data_path, folderID), 'r') as file:
        content = file.read()
        sub_files = content.split('<DOC>')
        sub_files = [file for file in sub_files if len(file) > 0]
        for file in sub_files:
            docID = data_extraction.extract_text_between_tags(file, doc_id_tag)
            text = data_extraction.extract_text_between_tags(file, doc_text_tag)
            words = data_extraction.tokenizer(text, stopwords_list)
            # stemming with Porter Stemmer
            stemmed_words = [ps.stem(word) for word in words]
            extracted_text[folderID][docID] = stemmed_words

# Summary of dataset

In [6]:
# Total sub docs
total_docids = []
total_sub_docs = 0
for key, _ in extracted_text.items():
    total_sub_docs += len(extracted_text[key])
    total_docids.extend(extracted_text[key].keys())

# Total tokens
tokens_list = []
for key, _ in extracted_text.items():
    for key, values in extracted_text[key].items():
        tokens_list.extend(values)
unique_tokens_list = sorted(list(set(tokens_list)))

print("No. of main docs in dataset : ", len(extracted_text.keys()))
print("No. of total sub documents in all main docs : ", total_sub_docs)
print("No. of total unique tokens in entire dataset : ", len(unique_tokens_list))

No. of main docs in dataset :  15
No. of total sub documents in all main docs :  5368
No. of total unique tokens in entire dataset :  32606


# Saving tokens and document ID in a text file

In [7]:
# Saving output
with open('../output/parser_output.txt', 'w') as file:
    for index in range(len(unique_tokens_list)):
        file.write(f'{index} : {unique_tokens_list[index]}\n')
    for index in range(len(total_docids)):
        file.write(f'{index} : {total_docids[index]}\n')